<a href="https://colab.research.google.com/github/tatsath/Interpretability/blob/main/sentiment_steering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install goodfire
!pip install openai
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.9 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall:

In [4]:
import goodfire

GOODFIRE_API_KEY = "sk-goodfire-dmKRWrIVThY26TCbxycGJjAO3q2gvxL2S3LBXuQk7ROn_qiJsk2CNA"
client = goodfire.Client(api_key=GOODFIRE_API_KEY)

In [5]:
variant = goodfire.Variant("meta-llama/Meta-Llama-3.1-8B-Instruct")

In [9]:
import pandas as pd

# df = pd.read_excel("sentence_sentiment_gpt4o.xlsx")
# df.head()

In [10]:
df = pd.read_excel('https://raw.githubusercontent.com/ashater/sae/master/sentence_sentiment_gpt4o.xlsx')

In [11]:
df.head()

,Version,Category,Sentiment,Sentence
0,v1,Revenue Growth,Very Positive,The company reported record-breaking revenue g...
1,v1,Revenue Growth,Very Positive,"Due to its innovative approach, the firm has s..."
2,v1,Debt Management,Very Positive,The early repayment of $2 billion in outstandi...
3,v1,Debt Management,Very Positive,The firm’s bonds have been upgraded to AAA sta...
4,v1,Market Performance,Very Positive,Investors are thrilled as the company’s stock ...


In [12]:
# prompt: get distinct columns from the Sentiment column of this df

import pandas as pd

df = pd.read_excel('https://raw.githubusercontent.com/ashater/sae/master/sentence_sentiment_gpt4o.xlsx')
distinct_sentiments = df['Sentiment'].unique()
distinct_sentiments


array(['Very Positive', 'Very Negative', 'Neutral', 'Somewhat Positive',
       'Somewhat Negative'], dtype=object)

In [13]:
df["Sentence"][0]

'The company reported record-breaking revenue growth of 40% this quarter, driven by robust sales and successful product launches.'

In [14]:
system_prompt = """\
You are an intelligent AI Assistant and your task is to provide a sentiment for the sentence provided.\
Reply with the sentiment only in one out of these five categories - 'Very Positive', 'Very Negative', 'Neutral', 'Somewhat Positive',
       'Somewhat Negative' . No explanation or "." is required.
"""

In [15]:
def get_model_response(prompt, model_variant, system_prompt=None):
    messages = [{"role": "user", "content": prompt}]
    try:
        response = client.chat.completions.create(
            messages=messages,
            model=model_variant,
            max_completion_tokens=50,
            system_prompt=system_prompt
        )
        return response.choices[0].message["content"].strip()
    except Exception as e:
        print(f"Error during completion: {e}")
        return "ERROR"


get_model_response(df["Sentence"][999], variant, system_prompt)

'Very Positive'

In [37]:
# Option 2 - steer on sentence category
query = "credit risk related"

features = client.features.search(query=query, model=variant, top_k=10)
features

FeatureGroup([
   0: "Financial credit and banking terminology",
   1: "Credit ratings and financial strength assessments",
   2: "Risk assessment in insurance underwriting",
   3: "Discussions of credit scores and credit history, especially concerns and improvement",
   4: "Business and financial risk assessment terminology",
   5: "Identity theft and financial fraud risks",
   6: "Trading risk warnings and responsible money management advice",
   7: "Bankruptcy and insolvency proceedings across languages",
   8: "The semantic root 'cred' relating to trust, authentication, and financial credit across languages",
   9: "Probabilistic financial loss estimation in risk assessment forms"
])

In [17]:
get_model_response(df["Sentence"][1], variant, system_prompt)

'Very Positive'

In [53]:
import pandas as pd
import random

# Assuming you have already loaded your DataFrame as 'df' and have the necessary functions

compared_df = pd.DataFrame(columns=['index', 'sentiment_annotated', 'sentence_not_steered', 'sentence_steered' ])

num_sentences_to_select = 50

for _ in range(num_sentences_to_select):
    random_index = random.choice(df.index)
    sentence = df["Sentence"][random_index]

    # Get non steered response
    variant.reset()
    response_not_steered = get_model_response(sentence, variant, system_prompt)

    # Reset steering and get non-steered response
    variant.reset()  # Reset steering for non-steered output
    # variant.reset()
    variant.set(features[1], 0)
    variant.set(features[7], .5)
    response_steered = get_model_response(sentence, variant, system_prompt)

    # Append results to DataFrame
    new_row = pd.DataFrame({
        'index': [random_index],
        'sentiment_annotated': [df['Sentiment'][random_index]],
        'sentence_not_steered': [response_not_steered],
        'sentence_steered': [response_steered],
    })
    compared_df = pd.concat([compared_df, new_row], ignore_index=True)
    compared_df['sentence_not_steered'] = compared_df['sentence_not_steered'].str.replace('.', '', regex=False)
    compared_df['sentence_steered'] = compared_df['sentence_steered'].str.replace('.', '', regex=False)

# Display the DataFrame
print(compared_df)

   index sentiment_annotated sentence_not_steered   sentence_steered
0   1638   Somewhat Negative    Somewhat Negative  Somewhat Negative
1   3814             Neutral              Neutral  Somewhat Positive
2   1001             Neutral              Neutral            Neutral
3   4626       Very Positive        Very Positive      Very Positive
4    110       Very Negative        Very Negative      Very Negative
5   5760   Somewhat Negative    Somewhat Negative  Somewhat Negative
6   3008   Somewhat Negative              Neutral  Somewhat Negative
7   3865   Somewhat Positive        Very Positive  Somewhat Positive
8   4363   Somewhat Positive        Very Positive  Somewhat Positive
9   2732   Somewhat Negative    Somewhat Negative  Somewhat Negative
10  1367             Neutral              Neutral            Neutral
11   597       Very Negative        Very Negative      Very Negative
12   958             Neutral        Very Positive            Neutral
13  5721       Very Negative      

In [54]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Before Steering
accuracy_before = accuracy_score(compared_df['sentiment_annotated'], compared_df['sentence_not_steered'])
precision_before = precision_score(compared_df['sentiment_annotated'], compared_df['sentence_not_steered'], average='weighted')  # Use 'weighted' for multi-class
recall_before = recall_score(compared_df['sentiment_annotated'], compared_df['sentence_not_steered'], average='weighted')
confusion_before = confusion_matrix(compared_df['sentiment_annotated'], compared_df['sentence_not_steered'])

# After Steering
accuracy_after = accuracy_score(compared_df['sentiment_annotated'], compared_df['sentence_steered'])
precision_after = precision_score(compared_df['sentiment_annotated'], compared_df['sentence_steered'], average='weighted')
recall_after = recall_score(compared_df['sentiment_annotated'], compared_df['sentence_steered'], average='weighted')
confusion_after = confusion_matrix(compared_df['sentiment_annotated'], compared_df['sentence_steered'])

In [55]:
metrics_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall'],
    'Before Steering': [accuracy_before, precision_before, recall_before],
    'After Steering': [accuracy_after, precision_after, recall_after]
})

In [56]:
print("Metrics:")
print(metrics_df)
print("\nConfusion Matrix (Before Steering):")
print(confusion_before)
print("\nConfusion Matrix (After Steering):")
print(confusion_after)

Metrics:
      Metric  Before Steering  After Steering
0   Accuracy         0.740000        0.860000
1  Precision         0.865873        0.885472
2     Recall         0.740000        0.860000

Confusion Matrix (Before Steering):
[[ 6  0  0  0  4]
 [ 1  7  0  0  0]
 [ 0  0  4  0  7]
 [ 0  1  0 13  0]
 [ 0  0  0  0  7]]

Confusion Matrix (After Steering):
[[ 7  0  2  0  1]
 [ 0  8  0  0  0]
 [ 0  0 11  0  0]
 [ 0  1  0 13  0]
 [ 0  0  3  0  4]]
